In [1]:
#import necessary dependecies
#https://github.com/manashpratim/Credit-Card-Fraud-Detection/blob/master/Credit_Card_Fraud_Detection.ipynb
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
#Set deirectory
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [103]:
#Data Path
data_path ='/content/drive/My Drive/Colab Notebooks/ZimnatInsurance/data/clean_data/'
save_path = '/content/drive/My Drive/Colab Notebooks/ZimnatInsurance/data/clean_data/'
soumission_path = '/content/drive/My Drive/Colab Notebooks/ZimnatInsurance/data/clean_data/'
trains='train_cl.csv'
tests='test_cl.csv'
submissions='soumission_path.csv'

In [104]:
#define some fxn to read the data set
def read_train():
    train =pd.read_csv(data_path + trains, sep=';', parse_dates=['join_date'])
    return train

def read_test():
    test =pd.read_csv(data_path + tests, sep=';', parse_dates=['join_date'])
    return test

#Assign defined fxns to variables
train = read_train();test=read_test()

In [105]:
#print all dataset shape
print(train.shape, test.shape)
train.head(3)

(611772, 11) (210000, 10)


,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0


## Future selection

In [106]:
import datetime as dt
#train = train[train['ID X PCODE'].notna()]
#train = train[train['join_date'].notna()]
#train['date'] = (train.join_date - train.join_date.min()).dt.days
#train = train.drop('join_date', axis=1)

# Convert time to float
train['join_date']=train['join_date'].map(dt.datetime.toordinal)
test['join_date']=test['join_date'].map(dt.datetime.toordinal)

# drop time 
#train = train.drop('join_date', axis=1)
#test = test.drop('join_date', axis=1)

In [107]:
x=train.drop(['Label', 'ID','ID X PCODE'],axis=1)
y=train['Label']
x.shape

(611772, 8)

## Split train data to train/test set

In [108]:
id = train['ID X PCODE']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)
print(x_train.shape)
print(x_test.shape)

(428240, 8)
(183532, 8)


# Test data

In [109]:
id_test = test['ID X PCODE']
test_ =test.drop(['ID','ID X PCODE'],axis=1)
print('test column',test.columns); print('test_  column', test_.columns)

test column Index(['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE', 'ID X PCODE'],
      dtype='object')
test_  column Index(['join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE'],
      dtype='object')


# Balanced train set

In [110]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42, ratio='not minority')
x_sm, y_sm = smote.fit_sample(x_train, y_train)

In [96]:
#Imbalanced label
unique, counts = np.unique(y_train, return_counts=True)
print('imbalanced label')
print( x_train.shape)
print('')
print(np.asarray((unique, counts)).T)


#Balanced label
unique_, counts_ = np.unique(y_sm, return_counts=True)
print('balanced label')
print(x_sm.shape)
print('')
print(np.asarray((unique_, counts_)).T)


imbalanced label
(428240, 8)

[[     0 381683]
 [     1  46557]]
balanced label
(428240, 8)

[[     0 381683]
 [     1  46557]]


## Logistic regression

In [ ]:
clf = LogisticRegressionCV(cv=10, random_state=42,class_weight='balanced',max_iter=1000)
clf.fit(x_sm, y_sm)
y_pred_prob = clf.predict_proba(x_test)
print('Average ROC_AUC score: {0:0.2f}'.format(roc_auc_score(y_test, y_pred_prob[:,1])))

Average ROC_AUC score: 0.77


In [ ]:
# Prediction and soumission
y_prob = clf.predict_proba(test_)
y_prob
summition = pd.DataFrame(y_prob[:,0], columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_clf.csv", index=False)
summition.head(n=3)

## Random Forest

In [ ]:
rfc=RandomForestClassifier(random_state=42,class_weight='balanced',n_estimators=200)
rfc.fit(x_sm, y_sm)
y_pred_proba = rfc.predict_proba(x_test)
print('Average ROC_AUC score: {0:0.2f}'.format(roc_auc_score(y_test, y_pred_proba[:,1])))

Average ROC_AUC score: 0.95


In [ ]:
# Soumission RF
y_prob = rfc.predict_proba(test_)
y_prob
summition = pd.DataFrame(y_prob[:,0], columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_rf.csv", index=False)
summition.head(n=3)

## XGBoost Classifier

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

data_dmatrix = xgb.DMatrix(data=x_sm,label=y_sm)

xgb = XGBClassifier(n_estimators=200,
                   objective= 'binary:logistic',
                   scale_pos_weight=1,
                   seed=42)
xgb.fit(x_sm, y_sm)
y_pred_prob = xgb.predict_proba(x_test)
print('Average ROC_AUC score: {0:0.2f}'.format(roc_auc_score(y_test, y_pred_prob[:,1])))

## Naive baive

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

nb = Pipeline([('tfidf', TfidfTransformer()),
               ('clf', BernoulliNB()),
              ])
nb.fit(x_sm, y_sm)
y_pred_prob_nb = nb.predict_proba(x_test)
print('Average ROC_AUC score: {0:0.2f}'.format(roc_auc_score(y_test, y_pred_prob_nb[:,1])))

Average ROC_AUC score: 0.52


In [ ]:
# Prediction
y_prob = nb.predict_proba(test_)
summition = pd.DataFrame(y_prob[:,0], columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_nb.csv", index=False)
summition.head(n=3)

# SVM

In [56]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

sgd = Pipeline([('clf', CalibratedClassifierCV(method = 'sigmoid',base_estimator=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-4, random_state=42, max_iter=100, tol=None))),
               ])

sgd.fit(x_sm, y_sm)
y_pred_prob_sgd = sgd.predict_proba(x_test)
print('Average ROC_AUC score: {0:0.2f}'.format(roc_auc_score(y_test, y_pred_prob_sgd[:,1])))

Average ROC_AUC score: 0.50


In [57]:
# Prediction and soumission
y_prob_svm = sgd.predict_proba(test_)
summition = pd.DataFrame(y_prob_svm[:,0], columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_sgd.csv", index=False)
summition.head(n=3)

,ID X PCODE,Label
0,F86J5PC X P5DA,0.891778
1,H6141K3 X P5DA,0.891778
2,RBAYUXZ X P5DA,0.891778


# Neural Network

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_sm, y_sm, epochs=5, batch_size=20)

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

In [66]:
# Prediction and soumission
y_prob_nn = model.predict_proba(test_)
summition = pd.DataFrame(y_prob_nn, columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_nn_3.csv", index=False)
summition.head(n=3)

,ID X PCODE,Label
0,F86J5PC X P5DA,0.111288
1,H6141K3 X P5DA,0.108495
2,RBAYUXZ X P5DA,0.113265


# Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

model_tf = keras.Sequential([
    keras.layers.Flatten(input_shape=(8,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
	  keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model_tf.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model_tf.fit(x_sm, y_sm, epochs=5, batch_size=1)
test_loss, test_acc = model_tf.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/5
763280/763280 [==============================] - 898s 1ms/step - loss: 38.6218 - accuracy: 0.5018
Epoch 2/5
763280/763280 [==============================] - 876s 1ms/step - loss: 0.6933 - accuracy: 0.5004
Epoch 3/5
763280/763280 [==============================] - 884s 1ms/step - loss: 0.6933 - accuracy: 0.4995
Epoch 4/5
763280/763280 [==============================] - 914s 1ms/step - loss: 0.6933 - accuracy: 0.4997
Epoch 5/5
5736/5736 [==============================] - 7s 1ms/step - loss: 0.6862 - accuracy: 0.8924
Test accuracy: 0.8923729658126831


In [ ]:
# Prediction and soumission
y_prob_tf = model_tf.predict(test_)
summition = pd.DataFrame(y_prob_tf, columns =['Label'])
summition['ID X PCODE'] = id_test
summition = summition[['ID X PCODE', 'Label']]
summition.to_csv(soumission_path +"soumission_tf.csv", index=False)
summition.head(n=3)

,ID X PCODE,Label
0,F86J5PC X P5DA,0.495533
1,H6141K3 X P5DA,0.495533
2,RBAYUXZ X P5DA,0.495533


## Grid cherche